# Composite Pattern
 The Composite Pattern allows you to compose objects into tree structures to represent part-whole hierarchies. Composite lets clients treat individual objects and compositions of objects uniformly.

In [7]:
from typing import List, Optional
from abc import ABC, abstractmethod

class Iterator(ABC):

    @abstractmethod
    def next(self) -> "MenuComponent":
        pass

    @abstractmethod
    def has_next(self) -> bool:
        pass

class MenuComponent:
    
    def get_name(self) -> str:
        raise ValueError("Unsupported Operation")
    
    def get_description(self) -> str:
        raise ValueError("Unsupported Operation")
    
    def get_price(self) -> int:
        raise ValueError("Unsupported Operation")
    
    def is_vegetarian(self) -> bool:
        raise ValueError("Unsupported Operation")
    
    def print(self) -> str:
        raise ValueError("Unsupported Operation")
    
    def add(self, component: "MenuComponent"):
        raise ValueError("Unsupported Operation")
    
    def remove(self, component: "MenuComponent"):
        raise ValueError("Unsupported Operation")
    
    def get_child(self, index: int):
        raise ValueError("Unsupported Operation")
    
    def create_iterator(self) -> Iterator:
        raise ValueError("Unsupported Operation")

class MenuComponentIterator(Iterator):
    def __init__(self, menu_components: List[MenuComponent]) -> None:
        self.menu_components = menu_components
        self.position = 0

    def has_next(self) -> bool:
        return self.position < len(self.menu_components)

    def next(self) -> MenuComponent:
        new_component = self.menu_components[self.position]
        self.position += 1
        return new_component
    
class NullIterator(Iterator):
    
    def has_next(self) -> bool:
        return False
        
    def next(self):
        raise ValueError("Unsupported Operation")


class CompositeIterator(Iterator):

    def __init__(self, iterator: Iterator) -> None:
        self.iterators: List[Iterator] = [iterator]

    def next(self) -> Optional[MenuComponent]:
        if self.has_next():
            component = self.iterators[-1].next()
            if isinstance(component, Menu):
                self.iterators.append(component.create_iterator())
            return component
        else:
            return None

    def has_next(self) -> bool:
        if len(self.iterators) == 0:
            return False
        else:
            if not self.iterators[-1].has_next():
                self.iterators.pop(-1)
                return self.has_next()
            else:
                return True



class MenuItem(MenuComponent):
    def __init__(self, name: str, description: str, vegetarian: bool, price: float) -> None:
        self.name = name
        self.description = description
        self.vegetarian = vegetarian
        self.price = price
        
    def get_name(self) -> str:
        return self.name
    
    def get_description(self) -> str:
        return self.description
    
    def get_price(self) -> float:
        return self.price
    
    def is_vegetarian(self) -> bool:
        return self.vegetarian
    
    def print(self) -> str:
        print(f"{self.get_name()} { '(v)' if self.is_vegetarian() else ''}, {self.get_price()}, -- {self.get_description()}")
        
    def create_iterator(self) -> Iterator:
        return NullIterator()
    
    

class Menu(MenuComponent):
    
    def __init__(self, name: str, description: str) -> None:
        self.menu_components : List[MenuComponent] = []
        self.name = name
        self.description = description
        
    def add(self, menu_component: MenuComponent) -> None:
        self.menu_components.append(menu_component)
        
    def remove(self, menu_component: MenuComponent) -> None:
        index = 0
        for menu in self.menu_components:
            if menu.get_name() == menu_component.get_name():
                self.menu_components.pop(index)
                break
            index += 1
    
    def get_child(self, i: int) -> MenuComponent:
        return self.menu_components[i]
    
    def get_name(self) -> str:
        return self.name
    
    def get_description(self) -> str:
        return self.description
    
    def print(self) -> None:
        print("")
        print(f"{self.get_name()}, {self.get_description()}")
        print("-----------------")
        
        for menu in self.menu_components:
            menu.print()
            
    def create_iterator(self) -> Iterator:
        return CompositeIterator(MenuComponentIterator(self.menu_components))
                
                
class Waitress:
    
    def __init__(self, all_menus: MenuComponent) -> None:
        self.menu_component = all_menus
        
    def print(self) -> None:
        """
        Print out the menu using internal iterators
        """
        self.menu_component.print()
        
    def print_vegetarian_menu(self) -> None:
        """
        Printing out the menu using external iterators rather than internal iterators from print
        """
        iterator = self.menu_component.create_iterator()
        print("")
        print("Vegetarian Menu")
        print("---")
        while iterator.has_next():
            menu_component = iterator.next()
            try:
                if menu_component.is_vegetarian():
                    menu_component.print()
            except ValueError:
                pass
                
                    
            
    


In [13]:
# Test Drive
pancake_house_menu = Menu("PANCAKE HOUSE MENU", "Breakfast")
diner_menu = Menu("DINER MENU", "Lunch")
cafe_menu = Menu("CAFE MENU", "Dinner")
dessert_menu = Menu("DESSERT MENU", "Dessert")

all_menus = Menu("ALL MENUS", "All menus combined")

all_menus.add(pancake_house_menu)
all_menus.add(diner_menu)
all_menus.add(cafe_menu)

diner_menu.add(MenuItem("Pasta", "Spaghetti with marinara Sauce", True, 3.89))
diner_menu.add(dessert_menu)
dessert_menu.add(MenuItem("Apple Pie", "Apple pie with flakey crust", True, 1.59))

waitress = Waitress(all_menus)
waitress.print()
print()
waitress.print_vegetarian_menu()



ALL MENUS, All menus combined
-----------------

PANCAKE HOUSE MENU, Breakfast
-----------------

DINER MENU, Lunch
-----------------
Pasta (v), 3.89, -- Spaghetti with marinara Sauce

DESSERT MENU, Dessert
-----------------
Apple Pie (v), 1.59, -- Apple pie with flakey crust

CAFE MENU, Dinner
-----------------


Vegetarian Menu
---
Pasta (v), 3.89, -- Spaghetti with marinara Sauce
Apple Pie (v), 1.59, -- Apple pie with flakey crust
Apple Pie (v), 1.59, -- Apple pie with flakey crust
